Cosas para procesamiento de imágenes:
* Evaluar características de la cámara
* Normalización (ver si los índices normalizan o estudiar si la normalización rompe el índice)
* Medir más características en el final, para discriminar de otra manera, ej: índice de textura

Informe breve que describa las acciones realizadas para todas las etapas, decisiones en cada una, resultados

## Preprocesamiento
Indices seleccionados:
* Cloudy days: CIVE
* Sunny days: ExG

Las partes quemadas de la imagen afectan en los índices (es algo esperado).<br>
Hay que probar las otras configuraciones de la cámara para detectar si las imágenes se queman.<br><br>
Conocimiento previo para el algoritmo:
* Cantidad de lineas en la imagen
* Estadio del cultivo para estimar ancho de cada linea
<br>

#### Aprendido
* Equalización de V en HSV rompe índices de vegetación (al menos los que trabajan con RGB)

#### Notas
* Cuando se haga el port, las imágenes binarizadas (uint8) cambiarlas a estructuras de Boolean

In [1]:
import cv2
import numpy as np
import os
import glob
import matplotlib
import matplotlib.pyplot as plt
import scipy
import peakutils # Buscar implementación en C++
import seaborn as sns # Para graficar boxplots
from ipynb.fs.full.Utils import *
from scipy.signal import savgol_filter # Buscar implementación en C++
from mpl_toolkits.mplot3d import axes3d # Para realizar gráficas 3D

In [40]:
reduction_to_see = 0.6 # Reducción del tamaño original para poder ver las imágenes en tamañas chicas
crop_width_bottom = 90 # Ancho de línea de cultivo en píxeles

* Recorte de ROI (80% inferior de la imagen para eliminar ruido generado en el punto de fuga)
* Calculo con índice CIVE
* Segmentación dinámica meidante el método de Otsu
* Reducción a un cuarto del tamaño original
* Obtención de porcentaje de vegetación en imágen
* Eroción sí el porcentaje de vegetación es mayor a 12%

In [41]:

path_to_imgs = 'img/test/'
file_name = 'test1'
_original = cv2.imread(path_to_imgs + file_name + '.png', cv2.IMREAD_COLOR)
if _original is None:
    print('Warning: imagen no cargada')
else:
# original, file_name = take_random_picture(path_to_imgs, 'test', 'jpg')
     mostrar_img(file_name,_original, reduction_to_see)
# Se hicieron pruebas con test4 o test5

In [42]:
# Cortar la parte superior de la imagen (esto se calcula de manera manual, depende del ángulo de pitch que tenga la cámara)

original = v_crop_top(_original, 0.25)
# mostrar_img(file_name,original, reduction_to_see)

In [5]:
# Índices posibles: cive exr veg exg exgr com1 com2

indice_seleccionado = 'exg'
index_img = img_to_color_index(original, indice_seleccionado)

In [6]:
# La convierto en binario de 8 bits para aplicar Otsu

amax = np.amax(index_img)
amin = np.amin(index_img)
scalingIndex = 255/(amax-amin)
gray_ind_img = ((index_img-amin)*scalingIndex)
gray_ind_img = gray_ind_img.astype(np.uint8)
list_to_invert = {'exg', 'exgr', 'veg', 'com1', 'com2'}
if (indice_seleccionado in list_to_invert):
    gray_ind_img = cv2.bitwise_not(gray_ind_img)
ret, segmented = cv2.threshold(
    gray_ind_img, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
# mostrar_imgs([file_name, indice_seleccionado + 'with otsu'],[original, segmented], reduction_to_see)
# cv2.imwrite(path_to_imgs + file_name + '_'+ indice_seleccionado +'.png', segmented)

In [7]:
# Apertura para eliminar ruido

kernel = np.ones((3,3), np.uint8)
segmented = cv2.morphologyEx(segmented, cv2.MORPH_OPEN, kernel, iterations=1)

In [8]:
# Utilizar el algoritmo con un reescalado de la imagen

rs_ratio = 0.25  # un cuarto de la imagen original
reducted = cv2.resize(segmented, None, fx=rs_ratio,
                      fy=rs_ratio, interpolation=cv2.INTER_AREA)
# mostrar_img('red', reducted)

In [9]:
# Apertura/eroción para eliminación de píxeles espurios o exceso de vegetación, denotando más las lineas
# Utilizo solo eroción para acentuar la diferencia entre líneas de cultivo

v_a = get_vegetation_amount(reducted)
print('Cantidad de vegetación original: ' + str(v_a))
if (v_a > 0.1):
    morph = (cv2.erode(reducted, kernel))
    v_a = get_vegetation_amount(morph)
    print('Cantidad de vegetación reducido: ' + str(v_a))
#     mostrar_imgs([file_name,'op1'], [reducted, morph])
else:
    morph = reducted.copy()

Cantidad de vegetación original: 0.13237413194444445
Cantidad de vegetación reducido: 0.01654513888888889


## Descriptores de línea de cultivo
* Histograma de la primera 12va franja horizontal de la imagen
* Suavizado con Savitzky-Golay, con ancho de ventana de 51 píxeles y polinomios orden 1
* Busqueda de máximos con distancia mínima de 100 píxeles (puntos de inicio)
* Calculo de líneas descriptoras mediante: <br>
a) Transformada de Hough con restricciónes<br>
b) (Consume demasiados recursos computacionales, hace falta comparar en recursos HW) Cruzamiento entre puntos de inicio con puntos de tope<br>
* Se generan grupos según las rectas que pase por +- 25 píxeles de cada punto de inicio
* Se selecciona una recta de cada grupo que sea la que mayor cantidad de vegación cruce

In [10]:
# Segmento que es analizado con los histogramas
# img_temp = morph[morph.shape[0] - h_space:morph.shape[0], 0:morph.shape[1]]
# mostrar_img('segmento bajo', img_temp)
# cv2.imwrite(path_to_imgs + file_name + '_' + 'segmentos' +'.png', img_temp)

In [11]:
# Calcular histograma de la franja inferior de la imágen, detectar la cantidad de picos según la cantidad
# de lineas se busquen, si hay más o menos de lo buscado se descarta la imagen.
# Luego de calculados los picos, calcular su centroide (la media de la distribución alrededor del pico)

#segment = seg_and[offset_seg_height:offset_seg_height + seg_height, 0:td_w]
h_space = int(morph.shape[0]/12)
horizontal_segment = reducted[morph.shape[0] -
                           h_space:morph.shape[0], 0:morph.shape[1]]
# print(horizontal_segment.shape)
histogram = row_histogram(horizontal_segment)

# Dibujar linea para mostrar sector analizado, y guarda
_morph_aux1 = cv2.merge((morph, morph, morph))
_morph_aux1 = cv2.line(_morph_aux1, (0, morph.shape[0]-h_space),
                  (morph.shape[1], morph.shape[0]-h_space), (0, 0, 255), 1)
# mostrar_img('line', _morph_aux1)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'segmentToInitialPoints' +'.png', _morph_aux1)

x = np.arange(0, morph.shape[1], 1)
# fig, ax = plt.subplots()
# ax.plot(x, histogram)
# ax.set(xlabel='Columna', ylabel='Cantidad de vegetación', title='Histograma de vegetación')
# ax.grid()
# fig.savefig('img/test/histogram_vegetation.png')
# plt.show()

In [12]:
# El tamaño de ventana depende del ancho de las lineas, se busca un valor de manera que suavice y resalte los picos de cultivo
# El grado del polinomio (por prueba) resulto ser mejor con grado 1, y eso puede ayudar en los calculos
# Buscar como implementarlo en c++ (Savitzky Golay filter)

smoothed_2dg = (savgol_filter(histogram, window_length=crop_width_bottom+1,
                              polyorder=1)).astype(np.int16)
# print(smoothed_2dg)

# fig, ax = plt.subplots()
# ax.plot(x, smoothed_2dg)
# ax.set(xlabel='Columna', ylabel='Cantidad de vegetación', title='Histograma de vegetación suavizado')
# ax.grid()
# fig.savefig('img/test/histogram_vegetation_smoothed.png')
# plt.show()

In [13]:
# Detección de picos, basados en una ventana de desplazamiento y punto de inflexión
# Los interpola

indexes = peakutils.indexes(
    smoothed_2dg, thres=0.05/max(smoothed_2dg), min_dist=100)

# fig, ax = plt.subplots()
# ax.plot(x, smoothed_2dg)
# ax.scatter(indexes, smoothed_2dg[indexes], c='blue')
# ax.set(xlabel='Columna', ylabel='Cantidad de vegetación', title='Máximos locales')
# fig.savefig('img/test/histogram_vegetation_smoothed+peaks.png')
# plt.show()

In [14]:
# Se calculan las rectas de Hough que salen de los puntos iniciales, que cumplan con las condiciones de ángulo
# verificar que otras condiciones hay

middle = morph.shape[0]-int(morph.shape[0]/2)
bottom_image = morph[middle:morph.shape[0], 0:morph.shape[1]]
# mostrar_img('bottom', bottom_image)

kernel_edge = np.array([[-2, 4, -2],  # Segmentos verticales
                        [-2, 4, -2],
                        [-2, 4, -2]])
edges = cv2.filter2D(bottom_image, -1, kernel_edge)
# mostrar_img('edges conv', edges)

In [15]:
print(morph.shape)

(360, 640)


In [16]:
# %%timeit -n 10 -r 10
# 67.8 ms ± 11.4 ms
# Busqueda de líneas con acumulación de vegetación
# Hay que reformular, puede dar errores ya que encuentra más vegetación a los costados
# Revisar si se puede mejorar y que supere a Hough en desempeño

# pixel_step = int(bottom_image.shape[1]/100)
# top_x_pos = np.zeros(len(indexes), np.uint16)
# veg_amount = np.zeros(len(indexes), np.uint16)
# current_x_top = 0
# y_top = 0
# y_bottom = bottom_image.shape[0]
# third_1 = (int)(bottom_image.shape[1]/3)
# third_2 = (int)(bottom_image.shape[1]*(2/3))
# while (current_x_top < bottom_image.shape[1]):
#     for x_bottom in range(0,len(indexes)):
#         x0, y0 = current_x_top, y_top
#         x1, y1 = indexes[x_bottom], y_bottom
#         m = -1 if ((x1-x0) == 0) else (y1-y0)/(x1-x0)
#         v_a = -1
#         if (m != -1):
#             b = (y0 - m * x0)
#             x = indexes[x_bottom]
#             if (x < third_1 and m < 0.49 and m > 0.2):
#                 v_a = get_vegetation_in_line(bottom_image, m, b)
#             if (x > third_2 and m > 2.65 and m < 2.92):
#                 v_a = get_vegetation_in_line(bottom_image, m, b)
#             if (x > third_1 and x < third_2 and (m > 2.75 or m < 0.39)):
#                 v_a = get_vegetation_in_line(bottom_image, m, b)
#         if veg_amount[x_bottom] < v_a:
#             veg_amount[x_bottom] = v_a
#             top_x_pos[x_bottom] = current_x_top
#     current_x_top = current_x_top + pixel_step

In [17]:
# %%timeit -n 10 -r 10
# 19.3 ms ± 525 µs
# Busqueda de líneas con Hough

# with_lines = cv2.merge((bottom_image, bottom_image, bottom_image))
lines = cv2.HoughLines(edges, 18, np.pi/90, 500)
third_1 = (int)(bottom_image.shape[1]/3)
third_2 = (int)(bottom_image.shape[1]*(2/3))
epsilon = (int)(0.01*bottom_image.shape[1])
delta = int(crop_width_bottom/2)
upper_limit = indexes + delta
lower_limit = indexes - delta
groups = {}
if type(lines) is np.ndarray:
    for line in lines:
        for rho, theta in line:
            if (np.sin(theta) != 0):
                m = -(np.cos(theta)/np.sin(theta))
                b = rho/np.sin(theta)
                x = int((bottom_image.shape[0]-b)/m)
                initial_points_condition = False
                group = -1
                for i in range(0, len(indexes)):
                    if (x <= upper_limit[i] and x >= lower_limit[i]):
                        group = i
                correct_angle_and_position = (x < third_1 and theta < 0.49 and theta > 0.2) or \
                    (x > third_2 and theta > 2.65 and theta < 2.92) or \
                    (x > third_1 and x < third_2 and (
                        theta > 2.75 or theta < 0.39))
                if (x > 0 and x < bottom_image.shape[1] and group != -1 and correct_angle_and_position):
                    v_a = get_vegetation_in_line(bottom_image, m, b)
#                     cv2.line(with_lines, (int(-(b/m)), 0), (int((with_lines.shape[0]-b)/m),
#                                                             with_lines.shape[0]), (255, 0, 0), 2)
                    if group not in groups.keys():
                        groups[group] = Line(rho, theta, v_a)
                    else:
                        if (groups[group].white_amount < v_a):
                            groups[group] = Line(rho, theta, v_a)

In [18]:
# Dibujar linea para mostrar sector analizado, y guarda

# _morph_aux1 = cv2.merge((morph, morph, morph))
# _morph_aux1 = cv2.line(_morph_aux1, (0, middle),
#                   (morph.shape[1], middle), (0, 0, 255), 1)
# for i in range(0, len(indexes)):
#     _morph_aux1 = cv2.line(
#         _morph_aux1, (indexes[i], 0), (indexes[i], morph.shape[0]), (255, 0, 0), 2)
#     _morph_aux1 = cv2.line(
#         _morph_aux1, (upper_limit[i], 0), (upper_limit[i], morph.shape[0]), (255, 255, 0), 2)
#     _morph_aux1 = cv2.line(
#         _morph_aux1, (lower_limit[i], 0), (lower_limit[i], morph.shape[0]), (255, 255, 0), 2)
# _morph_aux1 = _morph_aux1[_morph_aux1.shape[0] - h_space:_morph_aux1.shape[0], 0:_morph_aux1.shape[1]]
# mostrar_img('line', _morph_aux1)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'segmentToHough' +'.png', _morph_aux1)

In [19]:
# Para ver las rectas obtenidas por TH
with_lines = cv2.merge((bottom_image, bottom_image, bottom_image))
with_lines_2 = bottom_image.copy()
with_lines_2 = cv2.merge((with_lines_2,with_lines_2,with_lines_2))

# for index in range(0, len(indexes)):
#     cv2.line(with_lines_2, (top_x_pos[index], 0), (indexes[index], with_lines_2.shape[0]), (255,255,0), 2)

for key in groups.keys():
    line = groups.get(key)
    rho = line.rho
    theta = line.theta
    m = -(np.cos(theta)/np.sin(theta))
    b = rho/np.sin(theta)
    y0 = 0
    x0 = int((y0-b)/m)
    y1 = with_lines.shape[0]
    x1 = int((y1-b)/m)
    cv2.line(with_lines, (x0, y0), (x1, y1), (0, 255, 0), 2)

mostrar_img('Lines with hough', with_lines)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'lines' +'.png', with_lines)

### Cálculo de centroides
* Se utilizan doce segmentos, que se distribuyen cuatro en mitad baja y ocho en la parte alta

In [20]:
# Segmentación de la imagen en partes que disminuyen un 5% cuando se avanza hacia arriba

divisions = cv2.merge((morph, morph, morph))
total_y = morph.shape[0]
# El divisor (8) y la disminución (0.09) se obtuvieron de manera experimental
y_segments_bottom = np.arange(11,15) # Para obtener los 4 segmentos inferiores
y_segments_top = np.arange(0,9) # Para obtener los 8 segmentos superiores
# Botto: [384, middle], Top: [middle, 0]
y_segments_bottom = (np.multiply((np.true_divide(y_segments_bottom, np.sum(y_segments_bottom))), middle)).astype(np.uint16)
y_segments_top = (np.multiply((np.true_divide(y_segments_top, np.sum(y_segments_top))), middle)).astype(np.uint16)
y_segments_bottom = y_segments_bottom[::-1]
y_segments_top = y_segments_top[::-1]
y_segments_bottom[0] = y_segments_bottom[0] + middle - np.sum(y_segments_bottom)
y_segments_top[0] = y_segments_top[0] + middle - np.sum(y_segments_top)
y_segments = np.concatenate((y_segments_bottom[0:len(y_segments_bottom) - 1], y_segments_top[0:len(y_segments_top)]))
for i in range(len(y_segments) - 2, -1, -1):
    y_segments[i] = y_segments[i] + y_segments[i+1]
y_segments = np.concatenate(([morph.shape[0]], y_segments))

# # Para graficar
# for y in y_segments:
#     if y > 0:
#         if (y >= middle):
#             cv2.line(divisions, (0, y),
#                      (morph.shape[1], y), (255, 0, 0), 1)
#         else:
#             cv2.line(divisions, (0, y),
#                      (morph.shape[1], y), (0, 255, 0), 1)

# print('Una cantidad de ' + str(len(y_segments) - 1) + ' segmentos')
# print(y_segments)
# mostrar_img('segments', divisions)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'segmentos' +'.png', divisions)

In [21]:
# Para ver la primera franja con los segmentos de valores de cada línea de cultivo

# bottom_segment = morph[y_segments[1]:y_segments[0],:]
# bottom_segment = cv2.merge((bottom_segment, bottom_segment, bottom_segment))
# mostrar_img('bottom', bottom_segment)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'parte_baja' +'.png', bottom_segment)
# down_ind = indexes - int(crop_width_bottom / 2)
# up_ind = indexes + int(crop_width_bottom / 2)
# for i in range(0, len(indexes)):
#     cv2.line(bottom_segment, (down_ind[i], 0),
#                      (down_ind[i], bottom_image.shape[0]), (255, 255, 55), 2)
#     cv2.line(bottom_segment, (up_ind[i], 0),
#                      (up_ind[i], bottom_image.shape[0]), (255, 255, 55), 2)
#     cv2.line(bottom_segment, (indexes[i], 0),
#                      (indexes[i], bottom_image.shape[0]), (255, 0, 0), 2)
# mostrar_img('abc', bottom_segment)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'parte_baja_con_sectores' +'.png', bottom_segment)

In [22]:
# Dibujo de líneas para corroborar con los centroides que se calculan posteriormente

divisions_aux1 = divisions.copy()
img_offset = middle
for key in groups.keys():
    line = groups.get(key)
    rho = line.rho
    theta = line.theta
    m = -(np.cos(theta)/np.sin(theta))
    b = (rho/np.sin(theta)) + img_offset
    y0 = img_offset
    x0 = int((y0-b)/m)
    y1 = divisions_aux1.shape[0]
    x1 = int((y1-b)/m)
    cv2.line(divisions_aux1, (x0, y0), (x1, y1), (0, 255, 0), 2)
# mostrar_img('lineas', divisions_aux1)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'segmentos_con_TH' +'.png', divisions_aux1)

#### Cálculo de los centroides de la parte inferior de la imagen

In [23]:
# Se calcula el centro del ROI en base a las rectas encontradas por TH
# Si el microROI no tiene píxeles blancos se asume que el centro es el centroide
# Tres vacios consecutivos en una línea es el tope, sino se descarta la imagen -- a implementar

# Si se dibujan las rectas para corroborar centroides, comentar estas líneas
divisions_aux1 = divisions.copy()
img_offset = middle

x_centroids_matrix = np.zeros((len(indexes), len(y_segments) - 1), np.uint16)
y_centroids_matrix = np.zeros((len(indexes), len(y_segments) - 1), np.uint16)
crop_width_top = 20
crop_width = crop_width_bottom
crop_width_decay_seg = (crop_width_bottom - crop_width_top) / \
    (len(y_segments) - 1)  # (90-30)/12
# Ya que la recta está calculada para la parte inferior, es necesario un offset de b para transladarla
segment_index = 1
while y_segments[segment_index] >= middle:
    for key in groups.keys():
        # Obtener cada descriptor de línea
        line = groups.get(key)
        rho = line.rho
        theta = line.theta
        m = -(np.cos(theta)/np.sin(theta))
        b = (rho/np.sin(theta)) + img_offset
        y1, y2 = y_segments[segment_index -
                            1], y_segments[segment_index]  # Inferior, superior
        # El x del inferior, respecto a la recta
        x_temp = get_x_in_line(y1, m, b)
        # Desplazamientos según ancho de línea
        x1, x2 = x_temp - int(crop_width/2), x_temp + int(crop_width/2)
        # Pasarle microimagen con desplazamiento en x e y
        centroid = MicroROI(morph[y2:y1, x1:x2], x1, y2)
        x_centroids_matrix[key, segment_index - 1], y_centroids_matrix[key,
                                                                       segment_index - 1] = centroid.get_centroid()
        # Dibujar el centroide y la micro ROI
        cv2.line(divisions_aux1, (x1, y1), (x1, y2), (255, 255, 0), 1)
        cv2.line(divisions_aux1, (x2, y1), (x2, y2), (255, 255, 0), 1)
        cv2.circle(
            divisions_aux1, (x_centroids_matrix[key, segment_index - 1], y_centroids_matrix[key, segment_index - 1]), 3, (255, 0, 255), 2)
        key = key + 1  # Próxima línea
    # Nuevo ancho de microROI
    crop_width = crop_width - crop_width_decay_seg
    segment_index = segment_index + 1  # Próxima franja

# mostrar_img('divisiones y centroides', divisions_aux1)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'centroides' +'.png', divisions_aux1)
# print('pos x')
# print(x_centroids_matrix)
# print('pos y')
# print(y_centroids_matrix)

In [24]:
# Dibujar rectas obtenidas por mínimos cuadrados, en conjunto con los centroides obtenidos (los cuatro primeros)

# aux_img = divisions_aux1.copy()
# crop_width_aux = crop_width
# for line_index in range(0, x_centroids_matrix.shape[0]):  # Para cada línea
#     # Tomo elementos de [i, i+3] correspondientes a los últimos cuatro centroides
#     x_list = x_centroids_matrix[line_index, 0:4]
#     y_list = y_centroids_matrix[line_index, 0:4]
#     last_centroid = (
#         x_centroids_matrix[line_index, 3], y_centroids_matrix[line_index, 3])
#     m, b = least_squares_line(x_list, y_list, last_centroid)
#     y1, y2 = y_segments[segment_index -
#                         1], y_segments[segment_index]  # Inferior, superior
#     x1, x2 = get_x_in_line(aux_img.shape[0], m, b), get_x_in_line(y2, m, b)
#     cv2.line(aux_img, (x1, aux_img.shape[0]), (x2, y2), (0, 0, 255), 2)
#     x_temp = get_x_in_line(y1, m, b)
#     # Desplazamientos según ancho de línea
#     x1, x2 = x_temp - int(crop_width_aux/2), x_temp + int(crop_width_aux/2)
#     # Pasarle microimagen con desplazamiento en x e y
#     centroid = MicroROI(morph[y2:y1, x1:x2], x1, y2)
#     x_centroids_matrix[line_index, segment_index -
#                        1], y_centroids_matrix[line_index, segment_index-1] = centroid.get_centroid()
#     # Dibujar el centroide y la micro ROI
#     cv2.line(aux_img, (x1, y1), (x1, y2), (255, 255, 0), 1)
#     cv2.line(aux_img, (x2, y1), (x2, y2), (255, 255, 0), 1)
#     cv2.circle(aux_img, (x_centroids_matrix[line_index, segment_index-1],
#                          y_centroids_matrix[line_index, segment_index-1]), 3, (255, 0, 255), 2)

# mostrar_img('centroides y recta por mínimos', aux_img)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'centroide_y_TH' +'.png', divisions_aux1)

#### Cálculo de los centroides de la parte superior de la imagen

In [29]:
# Definir una función que me calcule la recta con mínimos cuadrados
# En y llevo el nivel actual, calculos la recta de cada línea con los últimos cuatro centroides
# Usar la recta con los últimos cuatro elementos para el ROI del siguiente nivel
# Calcula centroides del siguiente nivel y agregarlos en las matrices correspondientes
divisions_aux2 = divisions_aux1.copy()
# Continuo con el y de la celda anterior, para continuar con los segmentos correspondientes
last_centroids_index = 0
segment_index_aux = segment_index
crop_width_aux = crop_width
while segment_index_aux != (len(y_segments)):
    for line_index in range(0, x_centroids_matrix.shape[0]):  # Para cada línea
        # Tomo elementos de [i, i+3] correspondientes a los últimos cuatro centroides
        x_list = x_centroids_matrix[line_index,
                                    last_centroids_index:last_centroids_index + 4]
        y_list = y_centroids_matrix[line_index,
                                    last_centroids_index:last_centroids_index + 4]
        last_centroid = (
            x_centroids_matrix[line_index, last_centroids_index+3], y_centroids_matrix[line_index, last_centroids_index+3])

        m, b = least_squares_line(x_list, y_list, last_centroid)
        y1, y2 = y_segments[segment_index_aux -
                            1], y_segments[segment_index_aux]  # Inferior, superior
        x1, x2 = get_x_in_line(
            divisions_aux2.shape[0], m, b), get_x_in_line(y2, m, b)
#         cv2.line(divisions_aux2,
#                  (x1, divisions_aux2.shape[0]), (x2, y2), (0, 0, 255), 2)
        x_temp = get_x_in_line(y1, m, b)
        # Desplazamientos según ancho de línea
        x1, x2 = x_temp - int(crop_width_aux / 2), x_temp + \
            int(crop_width_aux / 2)
        # Pasarle microimagen con desplazamiento en x e y
        centroid = MicroROI(morph[y2:y1, x1:x2], x1, y2)
        x_centroids_matrix[line_index, segment_index_aux-1], y_centroids_matrix[line_index,
                                                                                segment_index_aux-1] = centroid.get_centroid()
        # Dibujar el centroide y la micro ROI
        cv2.line(divisions_aux2, (x1, y1), (x1, y2), (255, 255, 0), 1)
        cv2.line(divisions_aux2, (x2, y1), (x2, y2), (255, 255, 0), 1)
        cv2.circle(
            divisions_aux2, (x_centroids_matrix[line_index, segment_index_aux-1], y_centroids_matrix[line_index, segment_index_aux-1]), 3, (255, 0, 255), 2)
    segment_index_aux = segment_index_aux + 1  # Próxima franja
    crop_width_aux = crop_width_aux - crop_width_decay_seg
    last_centroids_index = last_centroids_index + 1

# print('x')
# print(x_centroids_matrix)
# print('y')
# print(y_centroids_matrix)
mostrar_img('centroides', divisions_aux2)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'centroides_hasta_arriba' +'.png', divisions_aux2)

#### Utilización de ajuste por mínimos cuadrados
* Cálculo de coeficientes de recta: mx+b
* Cálculo de coeficientes de cuadrática: ax^2+bx+c

In [30]:
# Una vez obtenidos todos los centroides de cada línea se realiza una análisis
# de regresión (se puede aplicar a cada conjunto de centroides y obtener los
# descriptores de la curva que mejor se adapte)
# Utiliza nuevamente mínimos cuadrados, y el calculo del resto para quedarse
# con la que mejor se ajuste (recta o cuadrática), se puede detectar la mejor por acumulación de vegetación
divisions_aux3 = divisions_aux2.copy()
y1, y2 = 0, divisions_aux3.shape[0]
y = np.arange(y1, y2)
# v_amount_line = 0

# Independiente y líneal
lineal_coef = np.zeros([x_centroids_matrix.shape[0], 2],
                       np.float16)
# Independiente, líneal y cuadrático por línea
quadratic_coef = np.zeros([x_centroids_matrix.shape[0], 3], np.float16)

for line_index in range(0, x_centroids_matrix.shape[0]):  # Para cada línea
    x_list = x_centroids_matrix[line_index, 0:x_centroids_matrix.shape[1]]
    y_list = y_centroids_matrix[line_index, 0:y_centroids_matrix.shape[1]]

    # Mínimos cuadrados para recta
    x_mean = int(mean(x_list))
    y_mean = int(mean(y_list))
    lineal_coef[line_index, 1], lineal_coef[line_index,
                                            0] = least_squares_line(x_list, y_list, (x_mean, y_mean))
#     v_amount_line = v_amount_line + \
#         get_vegetation_in_line(
#             morph, lineal_coef[line_index, 1], lineal_coef[line_index, 0])
    # Agregar a imagen
    x1, x2 = get_x_in_line(y1, lineal_coef[line_index, 1], lineal_coef[line_index, 0]), get_x_in_line(
        y2, lineal_coef[line_index, 1], lineal_coef[line_index, 0])
    cv2.line(divisions_aux3, (x1, y1), (x2, y2), (81, 139, 255), 2)

#     # Mínimos cuadrados para cuadrática
#     quadratic_coef[line_index, 2], quadratic_coef[line_index,
#                                                   1], quadratic_coef[line_index, 0] = least_squares_quadratic(x_list, y_list)
#     x1_quadratic, x2_quadratic = get_x_in_quadratic(
#         y, quadratic_coef[line_index, 2], quadratic_coef[line_index, 1], quadratic_coef[line_index, 0])
    # Guardar los x ya que los calculé, para no recalcular luego
#     x_quadratic = x1_quadratic if (
#         0 < x1_quadratic[0] and x1_quadratic[0] < divisions_aux3.shape[1]) else x2_quadratic
#     if (not np.isnan(x_quadratic).any()):
#         v_amount_quadratic = v_amount_quadratic + get_vegetation_in_quadratic(
#             morph, quadratic_coef[line_index, 2], quadratic_coef[line_index, 1], quadratic_coef[line_index, 0], x_quadratic)
# #         Agregar a imagen
#         pts = np.zeros((x_quadratic.shape[0], 2), np.uint64)
#         for i in range(0, x_quadratic.shape[0]):
#             pts[i, 0] = x_quadratic[i]
#             pts[i, 1] = y[i]
#         cv2.polylines(divisions_aux3, [pts], False, (0, 0, 255), 2)

# print(x_centroids_matrix[1,::])
# print(y_centroids_matrix[1,::])
# print("b: " + str(lineal_coef[1,0]) + " y m: " + str(lineal_coef[1,1]))
# print('Vegetación en línea: ' + str(v_amount_line))
# mostrar_img('total', divisions_aux3)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'cuadratica_minimos_cuadrados' +'.png', divisions_aux3)

### Filtrado de curvas
* Ya que solo se trabaja con rectas, se pospone estos filtros para luego

In [31]:
# Selección y verificación (reglas para verificar que sean rectas/cuadráticas
#válidas)
# 1) Regularidad en el espaciado entre líneas
# 2) No intersección dentro del ROI ni extensión por debajo de la imagen







#Salteo esta parte para luego, ya que las rectas encontradas con la imagen de prueba no tiene problemas

### Generación de máscaras con el uso de curvas descriptoras
* Actuamente solamente con rectas

In [32]:
# Máscaras según el ancho (90 píxeles a ojo, 20px arriba) de cultivo y de surco
# Solo estoy tomando en cuenta la recta, haría falta cambiar esto para utilizar la recta o la cuadrática según corresponda

addition = crop_width_bottom * 0.1  # Este porcentaje de ancho es dependiente del tipo de cultivo y el estadío en el que se encuentra

crop_mask = np.zeros(
    (divisions_aux3.shape[0], divisions_aux3.shape[1]), np.uint8)
furrow_crop_mask = crop_mask.copy()
y1, y2 = 0, crop_mask.shape[0]

crop_width = crop_width_bottom + addition
crop_width_decay = (crop_width_bottom - crop_width_top) / y2

furrow_width_bottom = int(get_crop_furrow_mean(np.sort(indexes)))
furrow_width = furrow_width_bottom
# Para calcular el espacio entre líneas en el tope
indexes_top = np.zeros(indexes.shape)
for i_line in range(0, len(lineal_coef)):
    indexes_top[i_line] = get_x_in_line(
        0, lineal_coef[i_line, 1], lineal_coef[i_line, 0])
furrow_width_top = int(get_crop_furrow_mean(np.sort(indexes_top)))
furrow_width_decay = (furrow_width_bottom - furrow_width_top) / y2

for y in range(y2 - 1, y1 - 1, -1):  # Desde la base la parte superior de la imagen
    for x in range(0, crop_mask.shape[1]):
        result = is_crop_or_furrow_lineal(
            lineal_coef, (x, y), crop_width, furrow_width)
        if (result == 1):
            crop_mask[y, x] = 255
        if (result == -1):
            furrow_crop_mask[y, x] = 255
    crop_width = crop_width - crop_width_decay
    furrow_width = furrow_width - furrow_width_decay

# mostrar_imgs(['máscara de cultivo', 'máscara de surco'],
#              [crop_mask, furrow_crop_mask])

In [33]:
# Utilizar las máscaras, luego de escalarlas al tamaño original, con AND a la máscara de vegetación
# _crop_mask = cv2.resize(crop_mask, None, fx=(1/rs_ratio),fy=(1/rs_ratio), interpolation=cv2.INTER_AREA)
# _furrow_crop_mask = cv2.resize(furrow_crop_mask, None, fx=(1/rs_ratio),fy=(1/rs_ratio), interpolation=cv2.INTER_AREA)

# cv2.imwrite(path_to_imgs + file_name + '_'+ 'mascara_cultivo' +'.png', _crop_mask)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'mascara_surco' +'.png', _furrow_crop_mask)

# _crop_mask = cv2.bitwise_and(_crop_mask, segmented)
# _furrow_crop_mask = cv2.bitwise_and(_furrow_crop_mask, segmented)

# Para la versión reducida
crop_mask_aux1 = cv2.bitwise_and(crop_mask, reducted)
furrow_crop_mask_aux1 = cv2.bitwise_and(furrow_crop_mask, reducted)
# mostrar_imgs(['máscara de cultivo', 'máscara de surco'],
#              [crop_mask_aux1, furrow_crop_mask_aux1])

# cv2.imwrite(path_to_imgs + file_name + '_'+ 'mascara_cultivo_veg' +'.png', crop_mask_aux1)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'mascara_surco_veg' +'.png', furrow_crop_mask_aux1)

### Prueba de distribución
* Hace falta aislar píxeles de maleza

In [34]:
# Preparación de datos para comparar distribución entre datos de crop y weed

# x_v = []
# y_v = []
# z_v = []
# x_w = []
# y_w = []
# z_w = []
# v_data = []
# w_data = []
# v_i = 0
# w_i = 0
# tope = 300000
# for x in range(0,segmented.shape[1]):
#     for y in range(0, segmented.shape[0]):
#         if (_crop_mask[y,x] == 255) and (v_i < tope):
#             v_i = v_i + 1
#             v_data.append(original[y,x])
#             x, y, z = original[y,x]
#             x_v.append(x)
#             y_v.append(y)
#             z_v.append(z)
# #             v_data.append(original[y,x])
#         if (_furrow_crop_mask[y,x] == 255) and (w_i < tope):
#             w_i = w_i + 1
#             w_data_data.append(original[y,x])
#             x, y, z = original[y,x]
#             x_w.append(x)
#             y_w.append(y)
#             z_w.append(z)
# #             w_data.append(original[y,x])
# x_v = np.array(x_v)
# y_v = np.array(y_v)
# z_v = np.array(z_v)
# x_w = np.array(x_w)
# y_w = np.array(y_w)
# z_w = np.array(z_w)
# v_data = np.array(v_data)
# w_data = np.array(w_data)

In [35]:
# fix, ax = plt.subplots(1,2, sharey=True)
# sns.boxplot(data=v_data, linewidth=2.5, palette='Set3', ax=ax[0])
# sns.boxplot(data=w_data, linewidth=2.5, palette='Set1', ax=ax[1])
# fig = plt.figure()
# ax1 = fig.add_subplot(111, projection='3d')
# ax1.scatter(x_v, y_v, z_v, c='g', marker='o')
# ax1.scatter(x_w, y_w, z_w, c='r', marker='o')
# plt.show()

## Clasificación en cuadrantes

In [36]:
# Tengo L: líneas y L+1: Surcos
# Sabiendo el espaciado entre líneas, puedo calcular en que surco estoy
# Tengo los índices de las franjas horizontales

# Aplicar apertura o erosiones sobre la máscara de surco
#kernel = np.ones((7,7),np.uint8)
#apertura = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
kernel_vertical = np.array([
    [0, 1, 0],
    [0, 1, 0],
    [0, 1, 0]
], np.uint8)
kernel_diagonal_1 = np.array([
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 0]
], np.uint8)
kernel_diagonal_2 = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
], np.uint8)
kernel_horizontal = np.array([
    [0, 0, 0],
    [1, 1, 1],
    [0, 0, 0]
], np.uint8)
kernel_cruz = np.array([
    [0, 1, 0],
    [1, 1, 1],
    [0, 1, 0]
], np.uint8)
kernel_x = np.array([
    [1, 0, 1],
    [0, 1, 0],
    [1, 0, 1]
], np.uint8)
_furrow_crop_mask_aperture = cv2.morphologyEx(
    furrow_crop_mask_aux1, cv2.MORPH_OPEN, kernel, iterations=1)
# for i in range(0, 0):
#     _furrow_crop_mask_aperture = cv2.morphologyEx(
#         _furrow_crop_mask_aperture, cv2.MORPH_OPEN, kernel_vertical, iterations=1)
#     _furrow_crop_mask_aperture = cv2.morphologyEx(
#         _furrow_crop_mask_aperture, cv2.MORPH_OPEN, kernel_diagonal_1, iterations=1)
#     _furrow_crop_mask_aperture = cv2.morphologyEx(
#         _furrow_crop_mask_aperture, cv2.MORPH_OPEN, kernel_diagonal_2, iterations=1)
#     _furrow_crop_mask_aperture = cv2.morphologyEx(
#         _furrow_crop_mask_aperture, cv2.MORPH_OPEN, kernel_horizontal, iterations=1)
mostrar_imgs(['orig', 'apert'], [furrow_crop_mask_aux1,
                                 _furrow_crop_mask_aperture])

# Gurdado para hacer comparativa
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'apertura1' +'.png', _furrow_crop_mask_aperture)

In [37]:
# Aplicar las máscaras de vegetación y de cultivo para clasificar visualmente
# _crop_mask_aux = cv2.resize(crop_mask, None, fx=(1/rs_ratio),fy=(1/rs_ratio), interpolation=cv2.INTER_AREA)
# _furrow_crop_mask_aux = cv2.resize(_furrow_crop_mask_aperture, None, fx=(1/rs_ratio),fy=(1/rs_ratio), interpolation=cv2.INTER_AREA)
# zeros = np.zeros(_crop_mask_aux.shape, np.uint8)
# mask = cv2.merge((zeros, _crop_mask_aux, _furrow_crop_mask_aux))
# total = cv2.add(original, mask)
# mostrar_imgs(['original', 'clasificada'], [original, total], reduction_to_see)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'clasificado' +'.png', total)

### Clasificación de maleza por sectores
* Se basa en la cantidad de vegetación en surco, luego de aplicar operaciones de eliminación de ruido

In [38]:
# Se puede disminuir el costo verificando solo en la parte interna de la máscara de maleza, hace falta conseguir puntos que pertenecen a ella
# Con whiles podría quitar varios saltos, comprobaciones y cálculos

__furrow_crop_mask = furrow_crop_mask.copy()
# Itero sobre cada franja
for y_index in range(0, len(y_segments) - 1):  # Itera 12 veces, una por segmento
    # En cada franja separo en cuadrantes separados por líneas de cultivo (cantidad de líneas + 1)
    # En cada cuadrante le sumo la cantidad de vegetación y el tamaño (tamaño tendrá 1 más para no dividir por 0)
    weed_amount = np.zeros(lineal_coef.shape[0] + 1, np.uint16)
    total_pxls = np.ones(lineal_coef.shape[0] + 1, np.uint16)
    x_by_line = np.zeros(lineal_coef.shape[0], np.uint16)
    # Obtengo los x de cada línea en la base de la franja, para realizar la separación por cuadrantes
    for i in range(0, lineal_coef.shape[0]):  # Itera #L veces
        x_by_line[i] = get_x_in_line(
            y_segments[y_index], lineal_coef[i, 1], lineal_coef[i, 0])
    # Itero en la franja correspondiente
    # Itera el alto de segmento (máx es 1/12 * (alto de imagen disminuida))
    for y in range(y_segments[y_index + 1], y_segments[y_index]):
        furrow_index = 0
        x = 0
        # Itera máx el ancho de la imagen
        while ((furrow_index < len(weed_amount)) and (x < crop_mask.shape[1])):
            # Me saco todos los 0's del comienzo, si es que tiene
            while ((x < crop_mask.shape[1]) and (furrow_crop_mask[y, x] == 0)):
                x = x + 1
            # Colóco el índice donde pertenece
            while ((furrow_index < x_by_line.shape[0]) and (x >= x_by_line[furrow_index])):
                furrow_index = furrow_index + 1
            while ((x < crop_mask.shape[1]) and (furrow_crop_mask[y, x] == 255)):
                # Procesamiento dentro de surco
                total_pxls[furrow_index] = total_pxls[furrow_index] + 1
                if _furrow_crop_mask_aperture[y, x] > 10:
                    weed_amount[furrow_index] = weed_amount[furrow_index] + 1
                x = x + 1
            furrow_index = furrow_index + 1
    # Adapto la máscara de surco acorde a si cada cuadrante pasa un porcentaje de maleza
    weed_porc = 0.08
    weed_amount = np.true_divide(weed_amount, total_pxls)
    for y in range(y_segments[y_index + 1], y_segments[y_index]):
        furrow_index = 0
        x = 0
        # Itera máx el ancho de la imagen
        while ((furrow_index < len(weed_amount)) and (x < crop_mask.shape[1])):
            # Me saco todos los 0's del comienzo, si es que tiene
            while ((x < crop_mask.shape[1]) and (furrow_crop_mask[y, x] == 0)):
                x = x + 1
            # Colóco el índice donde pertenece
            while ((furrow_index < x_by_line.shape[0]) and (x >= x_by_line[furrow_index])):
                furrow_index = furrow_index + 1
            if (furrow_index >= len(weed_amount)):
                print('fuera de rango y: ' + str(y) + ',x: ' +
                      str(x) + 'furr ind: ' + str(furrow_index))
            _data = 0 if (furrow_index < len(weed_amount)
                          and weed_amount[furrow_index] < weed_porc) else 255
            while ((x < crop_mask.shape[1]) and (furrow_crop_mask[y, x] == 255)):
                __furrow_crop_mask[y, x] = _data
                x = x + 1
            furrow_index = furrow_index + 1
#         weed_porc = weed_porc - weed_porc_red

# Sectorización de la máscara
__furrow_crop_mask_aux = __furrow_crop_mask.copy()
for y in y_segments:
    cv2.line(__furrow_crop_mask_aux, (0, y),
             (morph.shape[1], y), 255, 1)
# mostrar_img('furrow', __furrow_crop_mask_aux)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'Sectores' +'.png', __furrow_crop_mask_aux)

# Clasificación sectorizada
_crop_mask = cv2.resize(crop_mask, None, fx=(
    1/rs_ratio), fy=(1/rs_ratio), interpolation=cv2.INTER_AREA)
_furrow_crop_mask_aux = cv2.resize(__furrow_crop_mask, None, fx=(
    1/rs_ratio), fy=(1/rs_ratio), interpolation=cv2.INTER_AREA)
zeros = np.zeros(_crop_mask.shape, np.uint8)
mask = cv2.merge((zeros, _crop_mask, _furrow_crop_mask_aux))
total = cv2.add(original, mask)
mostrar_imgs(['original', 'clasificada'], [original, total], reduction_to_see)
# cv2.imwrite(path_to_imgs + file_name + '_'+ 'clasificado_sectores' +'.png', total)

True